In [3]:
import pandas as pd
from google.colab import files

def load_csv_file():
    print("Please upload your CSV file.")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded.")
        return None

    filename = next(iter(uploaded))
    print(f"Loading '{filename}' into DataFrame.")
    df = pd.read_csv(filename)

    return df

# Load the uploaded CSV file into a DataFrame
data = load_csv_file()

# Check if the data is loaded successfully
if data is not None:
    print("Data loaded successfully.")
    print(data.head())  # Show the first few rows of the DataFrame
else:
    print("Data loading failed.")

Please upload your CSV file.


Saving wiki_movie_plots_deduped.csv to wiki_movie_plots_deduped.csv
Loading 'wiki_movie_plots_deduped.csv' into DataFrame.
Data loaded successfully.
   Release Year                             Title Origin/Ethnicity  \
0          1901            Kansas Saloon Smashers         American   
1          1901     Love by the Light of the Moon         American   
2          1901           The Martyred Presidents         American   
3          1901  Terrible Teddy, the Grizzly King         American   
4          1902            Jack and the Beanstalk         American   

                             Director Cast    Genre  \
0                             Unknown  NaN  unknown   
1                             Unknown  NaN  unknown   
2                             Unknown  NaN  unknown   
3                             Unknown  NaN  unknown   
4  George S. Fleming, Edwin S. Porter  NaN  unknown   

                                           Wiki Page  \
0  https://en.wikipedia.org/wiki/Kansas_Sal

Explantion we loaded the csv file using the upload button

In [4]:
import pandas as pd

def process_data(filename, output_filename):
    # Load only the 'Title' and 'Plot' columns from the CSV file
    df = pd.read_csv(filename, usecols=['Title', 'Plot'])

    # Here you can apply any function to the dataframe if needed
    # For example, you could do some data cleaning or processing
    # processed_df = your_function(df)

    # For this example, we'll just use the original dataframe
    processed_df = df

    # Save the processed data to a new CSV file
    processed_df.to_csv(output_filename, index=False)
    print(f"Data processed and saved to '{output_filename}'.")

# Specify the name of the already uploaded file and the output file
input_filename = '/content/wiki_movie_plots_deduped.csv'
output_filename = '/content/processed_movies.csv'

# Process the data and save it to another file
process_data(input_filename, output_filename)

Data processed and saved to '/content/processed_movies.csv'.


Explanation: we have removed the extra columns from the dataset and kept "title" and "plot" columns

In [5]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

we have installed the sentence transformer libraries

In [6]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/processed_movies.csv')

# Merge the 'Title' and 'Plot' columns into a new 'Combined' column
df['Combined'] = df['Title'] + ' ' + df['Plot']

# Save the updated DataFrame back to a new CSV file
df.to_csv('updated_dataset.csv', index=False)

df.head()

,Title,Plot,Combined
0,Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr...",Kansas Saloon Smashers A bartender is working ...
1,Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov...","Love by the Light of the Moon The moon, painte..."
2,The Martyred Presidents,"The film, just over a minute long, is composed...","The Martyred Presidents The film, just over a ..."
3,"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...,"Terrible Teddy, the Grizzly King Lasting just ..."
4,Jack and the Beanstalk,The earliest known adaptation of the classic f...,Jack and the Beanstalk The earliest known adap...


In [9]:
%%capture
!pip install chromadb tqdm fireworks-ai python-dotenv pandas
!pip install sentence-transformers

In [10]:
import fireworks.client
import os
import dotenv
import chromadb
import json
from tqdm.auto import tqdm
import pandas as pd
import random
from google.colab import userdata

# you can set envs using Colab secrets
fireworks.client.api_key = userdata.get('FIREWORKS_API_KEY')

In [11]:
def get_completion(prompt, model=None, max_tokens=50):

    fw_model_dir = "accounts/fireworks/models/"

    if model is None:
        model = fw_model_dir + "llama-v2-7b"
    else:
        model = fw_model_dir + model

    completion = fireworks.client.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0
    )

    return completion.choices[0].text

In [12]:
mistral_llm = "mistral-7b-instruct-4k"

get_completion("Hello, my name is", model=mistral_llm)

' [Your Name]. I am a [Your Profession/Occupation]. I am writing to [Purpose of Writing].\n\nI am writing to [Purpose of Writing] because [Reason for Writing]. I believe that ['

In [13]:
prompt = """[INST]
Given the following wedding guest data, write a very short 3-sentences thank you letter:

{
  "name": "John Doe",
  "relationship": "Bride's cousin",
  "hometown": "New York, NY",
  "fun_fact": "Climbed Mount Everest in 2020",
  "attending_with": "Sophia Smith",
  "bride_groom_name": "Tom and Mary"
}

Use only the data provided in the JSON object above.

The senders of the letter is the bride and groom, Tom and Mary.
[/INST]"""

get_completion(prompt, model=mistral_llm, max_tokens=150)

" Dear John Doe,\n\nWe, Tom and Mary, would like to extend our heartfelt gratitude for your attendance at our wedding. It was a pleasure to have you there, and we truly appreciate the effort you made to be a part of our special day.\n\nWe were thrilled to learn about your fun fact - climbing Mount Everest is an incredible accomplishment! We hope you had a safe and memorable journey.\n\nThank you again for joining us on this special occasion. We hope to stay in touch and catch up on all the amazing things you've been up to.\n\nWith love,\n\nTom and Mary"

In [11]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load your dataset
file_path = '/content/processed_movies.csv'  # Update this with the path to your CSV file
df = pd.read_csv(file_path)

# Select only the 'Title' and 'Plot' columns (assuming your dataset has these columns)
data = df[['Title', 'Plot']].dropna()

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate embeddings
def generate_embeddings(texts):
    return model.encode(texts, show_progress_bar=True)

# Generate embeddings for the plots
plot_embeddings = generate_embeddings(data['Plot'].tolist())

# Function to find the most relevant plots based on a query
def get_relevant_plots(query, top_n=5):
    query_embedding = generate_embeddings([query])[0]
    similarities = cosine_similarity([query_embedding], plot_embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:top_n]
    return data.iloc[top_indices]

# Example query
input_query = input("enter any query")

# Get the top 5 relevant plots
relevant_plots = get_relevant_plots(input_query, top_n=5)
print(relevant_plots)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1091 [00:00<?, ?it/s]

enter any queryDocumentaries showcasing indigenous peoples' survival and daily life in Arctic regions


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                      Title                                               Plot
9101         The White Dawn  When three whalers become stranded in Northern...
7397   The Savage Innocents  Inuk, an Inuk, kills a priest who rejects his ...
438     Nanook of the North  The documentary follows the lives of an Inuk, ...
17227         Born in China  This documentary follows four individual anima...
18525     The Frozen Limits  A group of British pioneers decide to take par...


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : The White Dawn, The Savage innocents, Nanook of north, Born in China, The Forzen Limits. out of these five results, The White Dawn- RELEVANT, The savage innocents- IRRELEVANT, Born In china- IRRELEVANT, Nannook of north- relevant, the forzen limits - IRRELEVANT

Recall@1 = 1/(1+1)= 0.5

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are : The White Dawn, The Savage innocents, Nanook of north, Born in China, The Forzen Limits For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [12]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load your dataset
file_path = '/content/processed_movies.csv'  # Update this with the path to your CSV file
df = pd.read_csv(file_path)

# Select only the 'Title' and 'Plot' columns (assuming your dataset has these columns)
data = df[['Title', 'Plot']].dropna()

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate embeddings
def generate_embeddings(texts):
    return model.encode(texts, show_progress_bar=True)

# Generate embeddings for the plots
plot_embeddings = generate_embeddings(data['Plot'].tolist())

# Function to find the most relevant plots based on a query
def get_relevant_plots(query, top_n=5):
    query_embedding = generate_embeddings([query])[0]
    similarities = cosine_similarity([query_embedding], plot_embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:top_n]
    return data.iloc[top_indices]

# Example query
input_query = input("enter any query")

# Get the top 5 relevant plots
relevant_plots = get_relevant_plots(input_query, top_n=5)
print(relevant_plots)


Batches:   0%|          | 0/1091 [00:00<?, ?it/s]

enter any queryWestern romance


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                   Title  \
27407                           Kalgejje   
33373  A Saloon Wet with Beautiful Women   
18014       Age Jodi Jantam Tui Hobi Por   
31043               Mudhal Kadhal Mazhai   
32447                           Graduate   

                                                    Plot  
27407  The film is about romance and has music as its...  
33373                              An erotic romance.[4]  
18014  The film is mainly based on the rural backgrou...  
31043  This film is based on city based romance betwe...  
32447  This film is completely youth centric with rom...  


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : Kalgejje, A Saloon Wet with Beautiful Women, Age Jodi Jantam Tui Hobi Por, Mudhal Kadhal Mazhai, Graduate  out of these five results,  Kalgejje- RELEVANT, A Saloon Wet with Beautiful Women - IRRELEVANT, Age Jodi Jantam Tui Hobi Por - IRRELEVANT,  Mudhal Kadhal Mazhai- Irrelevant,  Graduate - RELEVANT

Recall@1 = 1/(1+1)= 0.5

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are :Kalgejje, A Saloon Wet with Beautiful Women, Age Jodi Jantam Tui Hobi Por, Mudhal Kadhal Mazhai, Graduate  For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [13]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load your dataset
file_path = '/content/processed_movies.csv'  # Update this with the path to your CSV file
df = pd.read_csv(file_path)

# Select only the 'Title' and 'Plot' columns (assuming your dataset has these columns)
data = df[['Title', 'Plot']].dropna()

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate embeddings
def generate_embeddings(texts):
    return model.encode(texts, show_progress_bar=True)

# Generate embeddings for the plots
plot_embeddings = generate_embeddings(data['Plot'].tolist())

# Function to find the most relevant plots based on a query
def get_relevant_plots(query, top_n=5):
    query_embedding = generate_embeddings([query])[0]
    similarities = cosine_similarity([query_embedding], plot_embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:top_n]
    return data.iloc[top_indices]

# Example query
input_query = input("enter any query")

# Get the top 5 relevant plots
relevant_plots = get_relevant_plots(input_query, top_n=5)
print(relevant_plots)


Batches:   0%|          | 0/1091 [00:00<?, ?it/s]

enter any querySilent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                      Title                                               Plot
293                  Sahara  Silent film femme fatale, Louise Glaum, portra...
5036   You're My Everything  Boston, 1924: A starstruck Hannah Adams waits ...
21941         Night Terrors  A young girl travels to Cairo to visit her fat...
22525        Shanghai Story  The film follows the rise and fall of a family...
28686  Mee Sindhutai Sapkal  The film is a biographical account of Sindhuta...


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : Sahara, you're my everuthingBoston, night terrors, shanghai story, mee sindhutai sapkal, out of these five results, T

Recall@1 = 1/(1+2)= 0.33

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are :  Sahara, you're my everuthingBoston, night terrors, shanghai story, mee sindhutai sapkal For this query, the reciporcal rank is 1/3 and MRR=3

In [14]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load your dataset
file_path = '/content/processed_movies.csv'  # Update this with the path to your CSV file
df = pd.read_csv(file_path)

# Select only the 'Title' and 'Plot' columns (assuming your dataset has these columns)
data = df[['Title', 'Plot']].dropna()

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate embeddings
def generate_embeddings(texts):
    return model.encode(texts, show_progress_bar=True)

# Generate embeddings for the plots
plot_embeddings = generate_embeddings(data['Plot'].tolist())

# Function to find the most relevant plots based on a query
def get_relevant_plots(query, top_n=5):
    query_embedding = generate_embeddings([query])[0]
    similarities = cosine_similarity([query_embedding], plot_embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:top_n]
    return data.iloc[top_indices]

# Example query
input_query = input("enter any query")

# Get the top 5 relevant plots
relevant_plots = get_relevant_plots(input_query, top_n=5)
print(relevant_plots)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1091 [00:00<?, ?it/s]

enter any queryComedy film, office disguises, boss's daughter, elopement


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                                Title  \
192    Amarilly of Clothes-Line Alley   
29139                Manamagan Thevai   
253                        Ask Father   
27475               Sweety Nanna Jodi   
6555                  He Laughed Last   

                                                    Plot  
192    Set in San Francisco during the early 1900s, t...  
29139  The movie was a musical comedy, narrating the ...  
253    Lloyd is a serious young middle-class guy on t...  
27475  This film is about a corporate love between th...  
6555   A Runyonesque Roaring 20s musical comedy about...  


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : Amarilly of Clothes-Line Alley, Manamagan Thevai, Ask Father, Sweety Nanna Jodi, He Laughed Last  out of these five results, T

Recall@1 = 1/(1+2)= 0.33

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are :Amarilly of Clothes-Line Alley, Manamagan Thevai, Ask Father, Sweety Nanna Jodi, He Laughed Last  For this query, the reciporcal rank is 1/3 and MRR=3

In [15]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load your dataset
file_path = '/content/processed_movies.csv'  # Update this with the path to your CSV file
df = pd.read_csv(file_path)

# Select only the 'Title' and 'Plot' columns (assuming your dataset has these columns)
data = df[['Title', 'Plot']].dropna()

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate embeddings
def generate_embeddings(texts):
    return model.encode(texts, show_progress_bar=True)

# Generate embeddings for the plots
plot_embeddings = generate_embeddings(data['Plot'].tolist())

# Function to find the most relevant plots based on a query
def get_relevant_plots(query, top_n=5):
    query_embedding = generate_embeddings([query])[0]
    similarities = cosine_similarity([query_embedding], plot_embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:top_n]
    return data.iloc[top_indices]

# Example query
input_query = input("enter any query")

# Get the top 5 relevant plots
relevant_plots = get_relevant_plots(input_query, top_n=5)
print(relevant_plots)


Batches:   0%|          | 0/1091 [00:00<?, ?it/s]

enter any queryLost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                      Title                                               Plot
162               Cleopatra  Because the film has been lost, the following ...
5979    Serpent of the Nile  The film opens in 44 BC, just after the assass...
18699  Caesar and Cleopatra  In this philosophical coming-of-age film, an a...
28411             Cleopatra  Cleopatra is an emotional family film which co...
19711         Carry On Cleo  The film opens during Caesar's invasions of Br...


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are: Cleopatra, Serpent of the nile, caesar and Cleopatra, cleopatra, carry on cleo. out of these five results,

Recall@1 = 1/(1+2)= 0.33

The metric is useful when we our system to return the best relevant item and want that item to be at higher position.For this query Cleopatra, Serpent of the nile, caesar and Cleopatra, cleopatra, carry on cleo, the reciporcal rank is 1/1 and MRR=1

In [16]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load your dataset
file_path = '/content/processed_movies.csv'  # Update this with the path to your CSV file
df = pd.read_csv(file_path)

# Select only the 'Title' and 'Plot' columns (assuming your dataset has these columns)
data = df[['Title', 'Plot']].dropna()

# Initialize the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate embeddings
def generate_embeddings(texts):
    return model.encode(texts, show_progress_bar=True)

# Generate embeddings for the plots
plot_embeddings = generate_embeddings(data['Plot'].tolist())

# Function to find the most relevant plots based on a query
def get_relevant_plots(query, top_n=5):
    query_embedding = generate_embeddings([query])[0]
    similarities = cosine_similarity([query_embedding], plot_embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:top_n]
    return data.iloc[top_indices]

# Example query
input_query = input("enter any query")

# Get the top 5 relevant plots
relevant_plots = get_relevant_plots(input_query, top_n=5)
print(relevant_plots)


Batches:   0%|          | 0/1091 [00:00<?, ?it/s]

enter any queryDenis Gage Deane-Tanner


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                       Title  \
9104            The Wrestler   
12127   Another Midnight Run   
67           Captain Alvarez   
5903   Last of the Comanches   
355            The Skywayman   

                                                    Plot  
9104   Professional wrestling promoter Frank Bass (Ed...  
12127  Jack Walsh (McDonald) is hired by bail bondsma...  
67     A melodrama about an American who becomes a re...  
5903   Sgt. Matt Trainor (Broderick Crawford) leads t...  
355    Captain Norman Craig (Ormer Locklear) returns ...  


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : The Wrestler, Another Midnight Run, Captain Alvarez, Last of the Comanches,The Skywayman. out of these five results, T

Recall@1 = 1/(1+2)= 0.33

The metric is useful when we our system to return the best relevant item and want that item to be at higher position. The top 5 results for this query are : The Wrestler, Another Midnight Run, Captain Alvarez, Last of the Comanches,The Skywayman For this query, the reciporcal rank is 1/1 and MRR=1